### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete_df = pd.merge(student_data, school_data, how="inner", on=["school_name", "school_name"])

school_data_complete_df

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90,14,Charter,1635,1043130
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70,14,Charter,1635,1043130
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84,14,Charter,1635,1043130
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90,14,Charter,1635,1043130


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
#summary calculations
school_count = school_data_complete_df["school_name"].nunique()
student_count = school_data_complete_df["student_name"].count()
#total_budget = school_data_complete_df["budget"].mean()
math_average = school_data_complete_df['math_score'].mean()
read_average = school_data_complete_df['reading_score'].mean()

school_data_complete_df['Total Schools'] = school_count
school_data_complete_df['Total Students'] = student_count
#school_data_complete_df['Total Budget'] = total_budget
school_data_complete_df['Average Math Score'] = math_average
school_data_complete_df['Average Reading Score'] = read_average

#formatting
#school_data_complete_df['Total Students'] = school_data_complete_df['Total Students'].map("{:,}".format)

school_data_complete_df.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,Total Schools,Total Students,Average Math Score,Average Reading Score
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,15,39170,78.985371,81.87784
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635,15,39170,78.985371,81.87784
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635,15,39170,78.985371,81.87784
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635,15,39170,78.985371,81.87784
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635,15,39170,78.985371,81.87784


In [3]:
school_data_complete_df.dtypes

Student ID                 int64
student_name              object
gender                    object
grade                     object
school_name               object
reading_score              int64
math_score                 int64
School ID                  int64
type                      object
size                       int64
budget                     int64
Total Schools              int64
Total Students             int64
Average Math Score       float64
Average Reading Score    float64
dtype: object

In [4]:
#total budget

unique_budget = school_data_complete_df['budget'].unique()


total_budget = unique_budget.sum()
#total_budget = '$' + unique_budget.sum().round(2).astype(str) + '.00'

total_buget = str(total_budget)


#type(total_budget)

print(total_budget)
print(unique_budget)
type(total_budget)

24649428
[1910635 1884411 1056600 3022020  917500 1319574 1081356 3124928  248087
  585858 1049400 2547363 3094650 1763916 1043130]


numpy.int64

In [5]:
# % passing math
pass_math_df = school_data_complete_df.loc[school_data_complete_df["math_score"] >= 70, :    ]
pass_math_count = pass_math_df['math_score'].count()
math_pass = (pass_math_count / student_count * 100).round(6)

In [6]:
# % passing reading
pass_read_df = school_data_complete_df.loc[school_data_complete_df['reading_score'] >= 70, :   ]
pass_read_count = pass_read_df['reading_score'].count()
read_pass = (pass_read_count / student_count * 100).round(6)

In [7]:
# % passing reading and math
pass_read_math_df = school_data_complete_df.loc[(school_data_complete_df['math_score'] >= 70) & 
                                                (school_data_complete_df['reading_score'] >= 70), :  ]
pass_read_math_count = pass_read_math_df['reading_score'].count()
pass_read_math = (pass_read_math_count / student_count * 100).round(6)

In [8]:
district_summary_df = pd.DataFrame({'Total Schools': [school_count], 
                                   'Total Student': [student_count],
                                   'Total Budget': total_buget,
                                    'Average Math Score': math_average,
                                    'Average Reading Score': read_average,
                                    '% Passing Math': math_pass,
                                    '% Passing Reading': read_pass,
                                    '% Overall Passing': pass_read_math,
                                   })

district_summary_df
#missing formatting for total student. missing total_budget formatting

,Total Schools,Total Student,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,39170,24649428,78.985371,81.87784,74.980853,85.805463,65.172326


In [9]:
district_summary_df.dtypes

Total Schools              int64
Total Student              int64
Total Budget              object
Average Math Score       float64
Average Reading Score    float64
% Passing Math           float64
% Passing Reading        float64
% Overall Passing        float64
dtype: object


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [10]:
school_split_df = school_data_complete_df

In [11]:
#create pass and fail math score bins
bins = [0, 69.9, 100]

group_labels = [0 , 1]

school_split_df["Math Pass/Fail"] = pd.cut(school_split_df['math_score'], bins, labels=group_labels)
school_split_df["Reading Pass/Fail"] = pd.cut(school_split_df['reading_score'], bins, labels=group_labels)

In [12]:
#casting 
school_split_df["Math Pass/Fail"] = school_split_df["Math Pass/Fail"].astype("int")
school_split_df["Reading Pass/Fail"] = school_split_df["Reading Pass/Fail"].astype("int")

In [13]:
# overall passing
school_split_df["Overall Pass/Fail"] = school_split_df["Reading Pass/Fail"] + school_split_df["Math Pass/Fail"]

In [15]:
#create df to filter for overall passing
overall_pass_df = school_split_df.loc[school_split_df["Overall Pass/Fail"] >= 2, : ]

In [24]:
#group overall pass by school
group_pass_df = overall_pass_df.groupby(["school_name"])

In [25]:
#group by school_name
group_df = school_split_df.groupby(["school_name"])

In [28]:
#testing
group_df["budget"].sum()

,budget
school_name,
Bailey High School,15549641728
Cabrera High School,2009159448
Figueroa High School,5557128039
Ford High School,4831365924
Griffin High School,1346890000
Hernandez High School,14007062700
Holden High School,105933149
Huang High School,5573322295
Johnson High School,14733628650


In [31]:
#testing
budget_school = group_df["budget"].sum()/group_df['Student ID'].count()

budget_school

school_name
Bailey High School       3124928.0
Cabrera High School      1081356.0
Figueroa High School     1884411.0
Ford High School         1763916.0
Griffin High School       917500.0
Hernandez High School    3022020.0
Holden High School        248087.0
Huang High School        1910635.0
Johnson High School      3094650.0
Pena High School          585858.0
Rodriguez High School    2547363.0
Shelton High School      1056600.0
Thomas High School       1043130.0
Wilson High School       1319574.0
Wright High School       1049400.0
dtype: float64

In [34]:
#creating columns for summary data table
reading_score_average = group_df["reading_score"].mean()
math_score_average = group_df['math_score'].mean()
total_students = group_df['Student ID'].count() 
percent_math_pass = (group_df["Math Pass/Fail"].sum() / group_df['Student ID'].count()) * 100
percent_read_pass = (group_df["Reading Pass/Fail"].sum() / group_df['Student ID'].count()) * 100
overall_pass = (group_pass_df['Overall Pass/Fail'].sum() / group_df['Student ID'].count())*100 / 2
budget_school = group_df["budget"].sum()/group_df['Student ID'].count()
budget_student = budget_school / total_students


In [35]:
school_summary_df = pd.DataFrame({'Total Students': total_students,
                                  'Total School Budget': budget_school,
                                  'Per Student Budget': budget_student,
                                 'Average Reading Score': reading_score_average,
                                 'Average Math Score': math_score_average,
                                 'Total Students': total_students,
                                 '% Passing Math': percent_math_pass,
                                 '# Passing Reading': percent_read_pass,
                                 '% Overall Passing': overall_pass})

school_summary_df.head()

,Total Students,Total School Budget,Per Student Budget,Average Reading Score,Average Math Score,% Passing Math,# Passing Reading,% Overall Passing
school_name,,,,,,,,
Bailey High School,4976,3124928.0,628.0,81.033963,77.048432,66.680064,81.933280,54.642283
Cabrera High School,1858,1081356.0,582.0,83.975780,83.061895,94.133477,97.039828,91.334769
Figueroa High School,2949,1884411.0,639.0,81.158020,76.711767,65.988471,80.739234,53.204476
Ford High School,2739,1763916.0,644.0,80.746258,77.102592,68.309602,79.299014,54.289887
Griffin High School,1468,917500.0,625.0,83.816757,83.351499,93.392371,97.138965,90.599455


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [ ]:
#head

## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [ ]:
#tail

## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

## Reading Score by Grade 

* Perform the same operations as above for reading scores

## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

## Scores by School Size

* Perform the same operations as above, based on school size.

## Scores by School Type

* Perform the same operations as above, based on school type